# Project thesis xgBoost

This is a notebook for testing package xgBoost (gradient boosting) with hyperparameteroptimization using optuna.

Import libraries to be used in notebook

In [26]:
import optuna
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import xgboost as xgb
from sklearn.metrics import mean_absolute_error
from scipy import stats
import shap
import random
import pyarrow.feather as feather
import matplotlib.pyplot as plt

Load morph and SNP data

In [27]:
#180K:
#data = pd.read_feather("C:/Users/gard_/Documents/MasterThesis/ProjectThesis/MyPipeline/Data/Processed/massBV.feather")

#70k:
data = pd.read_feather("C:/Users/gard_/Documents/MasterThesis/ProjectThesis/MyPipeline/Data/Processed/massBV_70k.feather")

In [4]:
# See that there are no repeated measurements for individuals
print(data["ringnr"].duplicated().any())
#print(massMorph["ringnr"].duplicated().any()

False


Here is how we load the CV-folds created in 'CreateCV', if we want to use those. The ringnrs are saved in a CSV-file 'cv_folds.csv'.

We want to create 10 sets of hyperparameters to be used in xgBoost 10-fold CV. See figure in RM for visualization. "Nested CV".

In [ ]:
# Load the CSV file
df = pd.read_csv('C:/Users/gard_/Documents/MasterThesis/ProjectThesis/MyPipeline/Data/CVfolds/cv_folds_mass_70k.csv')

Process data

X: Matrix of SNPs, y: Vector of pseudo-response (environmental effects removed)

Pseudo-response stored under 'ID'. See dataloader, code from Steffi how it is processed.

Here we used the predefined train/val/test sets from CreateCV. Setup is as follows:
Each run has training data 80%, validation and test 10%.
First run: testset = Fold 1 test, validation = Fold 2 test, trainset = Section 3-10 (i.e. Dataset - test Fold 1 - test Fold 2)
...Then increment validation and test set once each run until final run when:
testset = Fold 10 test, validation = Fold 1 test.

In [ ]:
#70k setup: Remove IID, 180k setup: remove FID. Else, everything is the same.
# X is ringnrs + all SNPs
X_CV = data.drop([
            "ID",
            "mean_pheno",
            "IID",
            "MAT",
            "PAT",
            "SEX",
            "PHENOTYPE",
            "hatchisland"
        ], axis = 1)

# Some of the SNPS have NA-values. Set to 0
X_CV = X_CV.fillna(0)
# Change from float to int64 for all columns not 'ringnr' (i.e. all SNPs)
X_temp = X_CV.drop(['ringnr'], axis = 1)
X_temp = X_temp.T.astype('int64').T
X_temp.insert(0, 'ringnr', X_CV['ringnr'])
X_CV = X_temp

# y is ringnrs + pseudo phenotype
y_CV = data[['ID', 'ringnr']]
y_CV_mean = data[['mean_pheno', 'ringnr']]

In [10]:
# Find val and test indices by creating boolean mask
test_idx = df[(df['Fold'] == 1) & (df['Set'] == 'test')]['ringnr'].values
val_idx = df[(df['Fold'] == 2) & (df['Set'] == 'test')]['ringnr'].values

In [11]:
# Training set equal to the intersect of training fold 1 and 2
f1 = df[((df["Fold"] == 1) & (df["Set"] == 'train'))]["ringnr"]
f2 = df[((df["Fold"] == 2) & (df["Set"] == 'train'))]["ringnr"]
intersect = set(f1).intersection(set(f2))
# Define training sets based on the intersection, then remove ringnrs
X_train = X_CV[X_CV["ringnr"].isin(intersect)].drop(["ringnr",], axis = 1)
y_train = y_CV[y_CV["ringnr"].isin(intersect)].drop(["ringnr",], axis = 1)
y_mean_train = y_CV_mean[y_CV_mean["ringnr"].isin(intersect)].drop(["ringnr",], axis = 1)

Create training-validation set, to be used when fitting model in testing. The train-val set is the whole training set excluding the testing-set (i.e. df training set Fold i_testset)

In [12]:
train_val_idx = df[(df['Fold'] == 1) & (df['Set'] == 'train')]['ringnr'].values # Same as test-idx (lower value)
X_train_val = X_CV[X_CV["ringnr"].isin(train_val_idx)].drop(["ringnr",],axis = 1)
y_train_val = y_CV[y_CV['ringnr'].isin(train_val_idx)].drop(["ringnr",], axis = 1)
y_mean_train_val = y_CV_mean[y_CV_mean['ringnr'].isin(train_val_idx)].drop(["ringnr",], axis = 1)

In [12]:
print(X_train.shape)
print(X_CV.shape)

(1529, 65238)
(3467, 65239)


In [13]:
y_val = y_CV[y_CV['ringnr'].isin(val_idx)]["ID"]
y_test = y_CV[y_CV['ringnr'].isin(test_idx)]["ID"]

y_mean_val = y_CV_mean[y_CV_mean['ringnr'].isin(val_idx)]["mean_pheno"]
y_mean_test = y_CV_mean[y_CV_mean['ringnr'].isin(test_idx)]["mean_pheno"]

X_val = X_CV[X_CV["ringnr"].isin(val_idx)].drop(["ringnr",], axis = 1)
X_test = X_CV[X_CV["ringnr"].isin(test_idx)].drop(["ringnr",], axis = 1)

We define an objective function to be optimized. Used by optuna.

See the following link for interpretation of hyperparameters:
https://forecastegy.com/posts/xgboost-hyperparameter-tuning-with-optuna/

n_estimators: Number of trees to be trained

learning_rate: How much each tree contributes to the final prediction, and how 'weak'/'strong' the learner is

max_depth: Maximum depth that a tree can grow to. Decides complexity of each tree in the model. Deeper tree: potentially capturing more complex patterns in the data, but risk overfitting training data.

subsample: Proportion [0,1] of the dataset to be randomly selected for training each tree. Controls amount of data used for bulding each tree in the model. Less data may combat overfitting.

colsample_bytree: Proportion [0,1] of features to be considered for each tree.

min_child_weight: Sets minimum sum of instance weights that must be present in a child node in each tree. In regression, this just means the number of observations that must be present in each node.

In [14]:
def objective(trial):
    """Objective function to be optimized by package optuna.
        Loss-function: MAE.
        n_jobs: Amount of processors used. Computer-specific, needs to be 
                changed according to computer."""
    params = {
        "objective": "reg:absoluteerror",
        "verbosity": 0,
        "n_estimators": 600, #trial.suggest_int("n_estimators", 50, 300),
        "learning_rate": trial.suggest_float("learning_rate", 1e-3, 0.1, log=True),
        "max_depth": trial.suggest_int("max_depth", 4, 14),
        "subsample": trial.suggest_float("subsample", 0.05, 1.0),
        "colsample_bytree": trial.suggest_float("colsample_bytree", 0.05, 1.0),
        "min_child_weight": trial.suggest_int("min_child_weight", 5, 25),
    }
    model = xgb.XGBRegressor(**params)
    model.fit(X_train, y_train, verbose=False)
    predictions = model.predict(X_val)
    mae = mean_absolute_error(y_val, predictions)
    return mae

To execute the optimization, we create a study object and pass the objective function to the optimize method.

The *direction* parameter specifies whether we want to minimize or maximize the objective function. The *n_trials* parameter defines the number of times the model will be trained with different hyperparameter values.

Study_full: Using full set of parameters

In [ ]:
study_full = optuna.create_study(direction='minimize')
study_full.optimize(objective, n_trials=20)

Once the optimization is complete, we can display the best hyperparameters and the RMSE score.

In [19]:
print('Best hyperparameters:', study_full.best_params)
print('Best MAE:', study_full.best_value)
#Full run:
#Best MAE: 0.8632183845614925, using 75/15/10%
# With best params: {'learning_rate': 0.007761168942941743, 'max_depth': 7, 'subsample': 0.8431936111543968, 'colsample_bytree': 0.47520937351444037, 'min_child_weight': 10}

Best hyperparameters: {'learning_rate': 0.01686051134623099, 'max_depth': 7, 'subsample': 0.24635348736268714, 'colsample_bytree': 0.44407330245285953, 'min_child_weight': 19}
Best MAE: 0.7832194356437325


Here the validation sets should be replaced by test sets. Keep in mind that the result is misleading as long as we are looking at correlation between predictions from training data and validation set. Validation set is used when optimizing hyperparameters.

In [17]:
model = xgb.XGBRegressor(n_estimators = 600, learning_rate = 0.01686051134623099, max_depth = 7, subsample = 0.24635348736268714, colsample_bytree = 0.44407330245285953, min_child_weight = 19)
model.fit(X_train_val, y_train_val, verbose = False)
predictions = model.predict(X_test)


In [18]:
res = stats.pearsonr(y_test, predictions)[0]
res

0.2896237404549282

Some plots from the hyperparameteroptimization.

In [45]:
#optuna.visualization.plot_optimization_history(study_full)

In [46]:
#optuna.visualization.plot_slice(study_full, params=["learning_rate","max_depth", "subsample", "colsample_bytree", "min_child_weight"])

Tip: If most of the best trials utilize a specific hyperparameter near the minimum or maximum value, consider expanding the search space for that hyperparameter.

For example, if most of the best trials use learning_rate close to 0.001, you should probably restart the optimization with the search space trial.suggest_float("learning_rate", 1e-4, 1e-2, log=True).

In [73]:
#optuna.visualization.plot_parallel_coordinate(study_full)

See the following link for visualization of the hyperparamters in the hyperparameterspace: https://www.youtube.com/watch?v=t-INgABWULw

# Extra:

In [ ]:
X_train_val, X_test, y_train_val, y_test = train_test_split(X, y, test_size=0.10, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train_val, y_train_val, test_size=0.1667, random_state=43)

In [ ]:
#Subset
X_train_val_sub, X_test_sub, y_train_val_sub, y_test_sub = train_test_split(X_subset, y, test_size=0.10, random_state=42)
X_train_sub, X_val_sub, y_train_sub, y_val_sub = train_test_split(X_train_val_sub, y_train_val_sub, test_size=0.1667, random_state=43)